# example: bypass command line api

In [1]:
!pip list|grep textattack

textattack                         0.2.3              


In [2]:
import textattack
import pickle
import pandas as pd
import torch

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
from collections import deque
num_remaining_attacks = 1
worklist = deque(range(0, num_remaining_attacks))
worklist_tail = worklist[-1]

In [4]:
# load model

In [5]:
from textattack.models.wrappers import SklearnModelWrapper
model = pickle.load(open('/home/qc/work/imdb_adversarial_attack/tf_idf_logistic_reg.pkl', 'rb'))
tokenizer = pickle.load(open('/home/qc/work/imdb_adversarial_attack/tf_idf_vectorizor2.pkl', 'rb'))
model_wrapper = SklearnModelWrapper(model, tokenizer)

In [6]:
# define attack

In [7]:
from textattack.attack_recipes import DeepWordBugGao2018

In [8]:
recipe = DeepWordBugGao2018(model_wrapper)

textattack: Unknown if model of class <class 'textattack.models.wrappers.sklearn_model_wrapper.SklearnModelWrapper'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [9]:
recipe

Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  CompositeTransformation(
    (0): WordSwapNeighboringCharacterSwap(
        (random_one):  True
      )
    (1): WordSwapRandomCharacterSubstitution(
        (random_one):  True
      )
    (2): WordSwapRandomCharacterDeletion(
        (random_one):  True
      )
    (3): WordSwapRandomCharacterInsertion(
        (random_one):  True
      )
    )
  (constraints): 
    (0): LevenshteinEditDistance(
        (max_edit_distance):  30
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
)

In [10]:
recipe.goal_function.model_batch_size = 32
recipe.goal_function.model_cache_size = 262144
recipe.constraint_cache_size = 262144

In [11]:
# get data set
_df = pd.read_csv('test_data.csv', header=None)
_df.columns = ['label', 'text']
dataset = _df[['text', 'label']].values.tolist()

In [12]:
# run things

In [13]:
results = []

In [14]:
for result in recipe.attack_dataset(dataset, indices=worklist):
    results.append(result)

In [15]:
print(results[0].__str__("ansi"))

1 (52%) --> 0 (51%)

Wall St. Bears Claw Back Into the Black ( Reuters ) . Reuters - Short - sellers , Wall Street 's dwindling band of ultra - cynics , are seeing green again .

Wall St. BeaSrs Claw Back Into the Black ( Reuters ) . Reuters - Short - sellers , Wall Street 's dwindling band of ultra - cynics , are seeing green again .


In [16]:
results[0].original_result.ground_truth_output

1

In [17]:
results[0].original_result.score

0.48340441672921686

In [18]:
results[0].original_result.attacked_text

<AttackedText "Wall St. Bears Claw Back Into the Black ( Reuters ) . Reuters - Short - sellers , Wall Street 's dwindling band of ultra - cynics , are seeing green again .">

In [19]:
results[0].perturbed_result.num_queries

29

In [20]:
results[0].perturbed_result.output

0

In [21]:
results[0].perturbed_result.score

0.510239840225168

In [22]:
results[0].perturbed_result.attacked_text.all_words_diff(results[0].original_result.attacked_text)

{2}